In [1]:
import tensorflow as tf
import random
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import time
import csv

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


# File Name setting

In [2]:
##tr 0 to 3 is location value
filename0_tr = "wifi_ET_0_1472570446"
filename1_tr = "wifi_ET_1_1472593839"
filename2_tr = "wifi_ET_2_1472598476"
filename3_tr = "wifi_ET_3_1472590170"

##te 0 to 3 is location value
filename0_te = "wifi_ET_0_1472546966"
filename1_te = "wifi_ET_1_1472554884"
filename2_te = "wifi_ET_2_1472551231"
filename3_te = "wifi_ET_3_1472564959"

# Variable setting

In [3]:
location0 = []
location1 = []
location2 = []
location3 = []

training_step = 500

# Constant variable

In [4]:
location = []

ap_address = []
ap_signal = []

bi_location = []
ap_lev = []
ap_lev_times = []
selected_ap = []

# Function ==================================

file은 한번 만 읽어서 각 array에 저장한 후에 처리하도록 함(같은 파일을 다시 읽지 않음)

In [5]:
def read_file(filename,ap_address, ap_signal, location):
    filename = "data/wifi_et/"+filename+".csv"
    with open(filename, 'rb') as csvfile:
        reader = csv.reader (csvfile, delimiter = ',')
        for row in reader:
            ap_address.append(row[2])
            ap_signal.append(row[3])
            location.append(row[4])

총 ap_address 들을 구해줌

In [6]:
def get_ap_lev(ap_address,ap_lev):
    for i in range(0,len(ap_address)):
        if ap_address[i] not in ap_lev:
            ap_lev.append(ap_address[i])
    return ap_lev

해당 ap가 총 몇 회 불렸는지 확인하는 함수 해당 함수는 get_ap_lev 함수 실행 이후에 돌려야함
count함수와 같음

In [7]:
def get_ap_levtimes(ap_address,ap_lev,ap_lev_times):
    ap_lev_times = np.zeros(len(ap_lev),dtype=np.int)
    for i in range(0, len(ap_address)):
        index = ap_lev.index(ap_address[i])
        ap_lev_times[index] += 1
    return ap_lev_times

ap_signal의 등장횟수가 많은 것을 기준으로 NN에 이용할 AP를 선택하는 함수

In [8]:
def select_ap_address(data_max,ap_lev, ap_lev_times, selected_ap):
    selected_ap_index = []
    threshold = data_max *0.90
    for i in range(0,len(ap_lev_times)):
        if ap_lev_times[i] >= threshold:
            selected_ap_index.append(i)
    for i in range(0, len(selected_ap_index)):
        selected_ap.append(ap_lev[selected_ap_index[i]])
    return threshold

data의 수가 가장 많은 것을 기준으로, 해당 data의 등장후 다음 등장 이전까지를 하나의 epoch으로 지정

위의 흐름대로 진행할 때 데이터 중에서 다른 location 데이터와 함께 들어와 있는 경우, 
무조건 행에 따라서 data를 나열하면 input(ap_signal)과 output(location)에 혼선이 있을 우려가 있음

[0 1 2 0 1 2 0 1] 로 등장하였다고 가정하고, 해당 하는 location 이<br>[5 5 5 5 5 5 5 4] 로 등장하였을 경우, 

마지막 두 개의 ap_address_index에서 0 1 이 다른 location인 경우가 발생되는 것을 방지하고자 함

우선 (location 별로 partition을 생성한 다음)==>raw data는 원래 떨어져 있으므로, 이것을 기준으로 다시 짜도록함.

각 파티션 별로 ap selected array를 구현한 다음 다시 합쳐지는 방식을 이용함


1. (선택된 ap index) x (data_max) 행렬
2. (location) x (data_max) 행렬

해당 함수는 selected_ap_signal의 갯수가 4개임을 알고서 하는 것.

수정이 요구되나,<br>
4개만 있어도 결과값이 잘 나온다면 애초에 셀렉트할때 4개만 해도 무방할 것이라고 판단됨.

In [9]:
def convert_to_selected_ap_array(ap_address,ap_signal,selected_ap,location, data_max):
    j = -1
    converted_ap_signal = np.zeros((data_max, len(selected_ap)))
    converted_location = []
    for i in range(0,len(ap_signal)):
        if ap_address[i] == selected_ap[0]:
            j = j+1
            converted_ap_signal[j][0] = ap_signal[i]
            converted_location.append(location[i])
        elif ap_address[i] == selected_ap[1]:
            converted_ap_signal[j][1] = ap_signal[i]
        elif ap_address[i] == selected_ap[2]:
            converted_ap_signal[j][2] = ap_signal[i]
        elif ap_address[i] == selected_ap[3]:
            converted_ap_signal[j][3] = ap_signal[i]
        else :
            continue
    return converted_ap_signal, converted_location

location을 softmax함수에 이용할 수 있도록 변환 <br>
1 => [0,1,0,0]<br>
3 => [0,0,0,1] 식으로 변환하는 함수

In [10]:
def make_location_to_binary_array(data_max,location):
    if data_max != len(location):
        print "Miss matched!"
        print data_max + "!=" + len(location)
        return 
    
    location_num = int(location[np.argmax(location)])
    bi_location = np.zeros((data_max,location_num+1), dtype = int)
    for i in range(0,data_max):
        bi_location[i,location[i]] = 1
    return bi_location

weight, bias 생성 함수 y = wx + b

In [11]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# ================================Function End

# Training data pre_processing

ap의 level과 총 level갯수

In [12]:
read_file(filename0_tr,ap_address, ap_signal, location0)
read_file(filename1_tr,ap_address, ap_signal, location1)
read_file(filename2_tr,ap_address, ap_signal, location2)
read_file(filename3_tr,ap_address, ap_signal, location3)

read_file(filename0_te,ap_address, ap_signal, location0)
read_file(filename1_te,ap_address, ap_signal, location1)
read_file(filename2_te,ap_address, ap_signal, location2)
read_file(filename3_te,ap_address, ap_signal, location3)
location = location0 + location1+ location2+ location3

get_ap_lev(ap_address,ap_lev)
ap_lev_times = get_ap_levtimes(ap_address,ap_lev,ap_lev_times)
data_max = ap_lev_times[np.argmax(ap_lev_times)]
threshold = select_ap_address(data_max,ap_lev, ap_lev_times, selected_ap)
converted_ap_signal, converted_location = convert_to_selected_ap_array(ap_address,ap_signal,selected_ap,location, data_max)
bi_location = make_location_to_binary_array(data_max, converted_location)

# Print training values

In [13]:
print get_ap_lev(ap_address, ap_lev)
print len(ap_lev)

['00:26:66:d0:c4:dc', '00:27:1c:ea:40:af', '00:23:aa:3d:d9:41', '00:27:1c:08:41:16', '00:01:36:2d:ea:4c', '00:40:5a:db:5e:c9', '08:5d:dd:71:40:b4', '00:07:89:36:e2:ba', '90:9f:33:40:a0:9e', '64:e5:99:6a:ec:24', '00:30:0d:62:4c:24', '00:26:66:78:99:a8', '00:27:1c:3b:0a:53', '00:23:aa:3c:7a:c1', '90:9f:33:f5:73:cc', '00:26:66:76:37:9c', '08:5d:dd:48:92:27', '04:8d:39:36:e5:9d', '08:5d:dd:89:c1:93', 'c8:3a:35:05:ae:e0', '90:9f:33:a5:15:82', '90:9f:33:6d:8e:b6', '64:e5:99:82:5a:21', '90:9f:33:8f:a2:6a', '08:5d:dd:0b:0a:3a', '00:07:89:2b:53:35', '00:25:42:e0:15:58']
27


In [14]:
print ap_lev_times
print len(ap_lev_times)

[2895 2882 1260 2777 2863  295  558  151  202  109  376  313   44   32  146
 1296    4  358  185  110   52  134  336  125    4   10    3]
27


In [15]:
print threshold
print selected_ap

2605.5
['00:26:66:d0:c4:dc', '00:27:1c:ea:40:af', '00:27:1c:08:41:16', '00:01:36:2d:ea:4c']


In [16]:
print len(converted_location)

2895


# Training data set up

In [17]:
print converted_ap_signal

[[-56. -53. -76. -81.]
 [-54. -55. -76. -88.]
 [-56. -55. -74. -87.]
 ..., 
 [-65. -84. -92. -83.]
 [-67. -86. -89. -87.]
 [-65. -76. -92. -87.]]


In [18]:
print bi_location

[[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 ..., 
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]]


In [19]:
tr_x = converted_ap_signal

In [20]:
tr_y = bi_location

# Variable reset

In [21]:
location0 = []
location1 = []
location2 = []
location3 = []
ap_signal = []
ap_address = []
location = []
ap_lev = []
ap_lev_times = []
data_max = 0

# Testing data set up

In [22]:
read_file(filename0_te,ap_address, ap_signal, location0)
read_file(filename1_te,ap_address, ap_signal, location1)
read_file(filename2_te,ap_address, ap_signal, location2)
read_file(filename3_te,ap_address, ap_signal, location3)
location = location0 + location1+ location2 + location3

get_ap_lev(ap_address,ap_lev)
ap_lev_times = get_ap_levtimes(ap_address,ap_lev,ap_lev_times)
data_max = ap_lev_times[np.argmax(ap_lev_times)]
converted_ap_signal, converted_location = convert_to_selected_ap_array(ap_address,ap_signal,selected_ap,location, data_max)
bi_location = make_location_to_binary_array(data_max, converted_location)

In [23]:
print converted_ap_signal

[[-48. -59. -59. -66.]
 [-49. -59. -60. -73.]
 [-50. -60. -60. -73.]
 ..., 
 [-65. -84. -92. -83.]
 [-67. -86. -89. -87.]
 [-65. -76. -92. -87.]]


In [24]:
print bi_location

[[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 ..., 
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]]


In [25]:
te_x = converted_ap_signal

In [26]:
te_y = bi_location

In [27]:
print selected_ap
print data_max

['00:26:66:d0:c4:dc', '00:27:1c:ea:40:af', '00:27:1c:08:41:16', '00:01:36:2d:ea:4c']
1431


# Neural network algorithm : 2 hidden layer, softmax

In [28]:
location_num = int(location[np.argmax(location)]) + 1 # 0, 1, 2, 3
selected_num = len(selected_ap)
hidden_dim = selected_num

In [29]:
print location_num
print selected_num

4
4


In [30]:
W = weight_variable([selected_num, hidden_dim])
b = bias_variable([hidden_dim])

W2 = weight_variable([hidden_dim,location_num])
b2 = bias_variable([location_num])

X = tf.placeholder("float",[None, selected_num])
Y = tf.placeholder("float",[None,location_num])

hidden = tf.nn.relu(tf.matmul(X, W)+b)
sfmax = tf.nn.softmax(tf.matmul(hidden,W2)+b2)

cross_entropy = -tf.reduce_sum(Y*tf.log(sfmax))
train = tf.train.AdamOptimizer(1e-2).minimize(cross_entropy)

In [31]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())

for step in range(training_step):
    sess.run(train, feed_dict = {X: tr_x, Y: tr_y})
    correct_prediction = tf.equal(tf.argmax(sfmax,1), tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print sess.run(accuracy,feed_dict = {X: te_x, Y: te_y})   

0.197764
0.385045
0.301887
0.2942
0.296296
0.29979
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.300489
0.30